# load model


In [7]:
%load_ext autoreload
%autoreload 2
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1" # select GPU

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


[autoreload of src.models.cav_mae failed: Traceback (most recent call last):
  File "/home/chenghao/anaconda3/envs/MAE_C/lib/python3.8/site-packages/IPython/extensions/autoreload.py", line 273, in check
    superreload(m, reload, self.old_objects)
  File "/home/chenghao/anaconda3/envs/MAE_C/lib/python3.8/site-packages/IPython/extensions/autoreload.py", line 471, in superreload
    module = reload(module)
  File "/home/chenghao/anaconda3/envs/MAE_C/lib/python3.8/importlib/__init__.py", line 169, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 604, in _exec
  File "<frozen importlib._bootstrap_external>", line 843, in exec_module
  File "<frozen importlib._bootstrap>", line 219, in _call_with_frames_removed
  File "/home/chenghao/Project/cav-mae/src/models/cav_mae.py", line 20, in <module>
    from models.modeling_finetune import vit_base_patch16_160
ModuleNotFoundError: No module named 'models'
]


# cav-mae 的整体模型定义并导入预训练权重

In [4]:
import os
import torch
import timm
from src.models import CAVMAE

assert timm.__version__ == '0.4.5' # it is important to have right version of timm

model_path = 'pretrained_model/audio_model.pth'
os.environ["CUDA_VISIBLE_DEVICES"] = "6,7" # select GPU
# CAV-MAE model with decoder
#print(torch.cuda.device_count())
torch.cuda.set_device(1)
audio_model = CAVMAE(audio_length=1024,
                     modality_specific_depth=11,
                     norm_pix_loss=True, tr_pos=False) # most models are trained with pixel normalization and non-trainabe positional embedding

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
mdl_weight = torch.load(model_path, map_location=device)
audio_model = torch.nn.DataParallel(audio_model) # it is important to convert the model to dataparallel object as all weights are saved in dataparallel format (i.e., in module.xxx)
miss, unexpected = audio_model.load_state_dict(mdl_weight, strict=False)
print('miss=',miss, 'unexpected=', unexpected) # check if all weights are correctly loaded

A CAV-MAE Model
Use norm_pix_loss:  True
Learnable Positional Embedding:  False
Number of Audio Patches: 512, Visual Patches: 196
Audio Positional Embedding Shape: torch.Size([1, 512, 768])
Visual Positional Embedding Shape: torch.Size([1, 196, 768])
miss= ['module.blocks_maedfer.patch_embed.proj.weight', 'module.blocks_maedfer.patch_embed.proj.bias', 'module.blocks_maedfer.blocks.0.norm1.weight', 'module.blocks_maedfer.blocks.0.norm1.bias', 'module.blocks_maedfer.blocks.0.attn.q_bias', 'module.blocks_maedfer.blocks.0.attn.v_bias', 'module.blocks_maedfer.blocks.0.attn.qkv.weight', 'module.blocks_maedfer.blocks.0.attn.proj.weight', 'module.blocks_maedfer.blocks.0.attn.proj.bias', 'module.blocks_maedfer.blocks.0.norm2.weight', 'module.blocks_maedfer.blocks.0.norm2.bias', 'module.blocks_maedfer.blocks.0.mlp.fc1.weight', 'module.blocks_maedfer.blocks.0.mlp.fc1.bias', 'module.blocks_maedfer.blocks.0.mlp.fc2.weight', 'module.blocks_maedfer.blocks.0.mlp.fc2.bias', 'module.blocks_maedfer.block

# 定义cav-mae模型，查看其中的video模块输出维度

In [3]:
import os
import torch
import timm
from src.models import CAVMAE

cav_mae_model = CAVMAE(audio_length=1024, 
                       modality_specific_depth=11,
                       norm_pix_loss=True, tr_pos=False)

A CAV-MAE Model
Use norm_pix_loss:  True
Learnable Positional Embedding:  False
Number of Audio Patches: 512, Visual Patches: 196
Audio Positional Embedding Shape: torch.Size([1, 512, 768])
Visual Positional Embedding Shape: torch.Size([1, 196, 768])


# 定义mae dfer模型

In [5]:
from src.models import modeling_finetune
import torch
from src.models.modeling_finetune import vit_base_patch16_160

# 定义模型
model1 = vit_base_patch16_160()


# 用maedfer 进行一次前向

In [6]:

# 创建随机输入数据
input_data = torch.rand(1, 3, 16, 160, 160)

# 进行前向传播
output1, output1_feature = model1(input_data,save_feature=True)
model1.reset_classifier( num_classes=10)
print(output1_feature.shape)
print(output1.shape)


torch.Size([1, 3, 16, 160, 160])
torch.Size([1, 3, 16, 160, 160])
torch.Size([1, 768])
torch.Size([1, 1000])


In [7]:
import torch
import torch.nn as nn

# 假设输入张量的形状是 (Batch_size, channel, 224, 224)
Batch_size = 4
channel = 3
original_tensor = torch.rand(Batch_size, channel, 224, 224)

# 首先将张量展平
flattened = original_tensor.view(Batch_size, -1)

# 定义一个全连接层，输出尺寸为 channel*160*160
fc = nn.Linear(in_features=channel*224*224, out_features=channel*160*160)

# 应用全连接层
output_tensor = fc(flattened)

# 将输出张量调整为 (Batch_size, channel, 160, 160)
output_tensor = output_tensor.view(Batch_size, channel, 160, 160)

# 输出结果张量的形状
print(output_tensor.shape)  # 应该是 (Batch_size, channel, 160, 160)


torch.Size([4, 3, 160, 160])
